In [31]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder, StandardScaler, KBinsDiscretizer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
from lightgbm import LGBMClassifier
from category_encoders import OneHotEncoder
from sklearn.model_selection import cross_val_predict
from warnings import filterwarnings
from sklearn.impute import SimpleImputer
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn import svm



In [46]:
filterwarnings('ignore')
import os
train = pd.read_csv("data/train_2.csv")
print("train shape", train.shape)
test = pd.read_csv("data/test_2.csv")
print("test shape", test.shape)


train["A_avg"] = (train["A_1"]+train["A_2"]) / 2
for i in range(165):
    train["XD_"+str(i+1)+str(i+2)] = (train["D_"+str(i+1)]+train["D_"+str(i+2)])/2 


train.fillna(value=-1)
test.fillna(value=-1)

target_column = "target"
id_column = "id"
categorical_cols = [c for c in test.columns if test[c].dtype in [np.object]]
numerical_cols = [c for c in test.columns if test[c].dtype in [np.float, np.int] and c not in [target_column, id_column]]
print("Number of features", len(categorical_cols)+len(numerical_cols))


train shape (400665, 202)
test shape (399335, 201)
Number of features 200


In [54]:
column_transformer = ColumnTransformer([
        ('num', StandardScaler(), numerical_cols),
        ('cat', OneHotEncoder(), categorical_cols)
    ])

In [55]:
classifier_lgbm = make_pipeline(
    column_transformer,
    
    LGBMClassifier(n_jobs=-1)   
)

In [52]:
classifier_xgb = make_pipeline(
    column_transformer,
    XGBClassifier(n_jobs=-1)   
)

In [16]:
classifier_rf = make_pipeline(column_transformer,RandomForestClassifier(random_state=42))
classifier_svm = make_pipeline(svm.SVC(column_transformer,random_state=42))

#pipe_lr_pca = make_pipeline(column_transformer, LogisticRegression(random_state=42))
#pipe_rf_pca = make_pipeline(column_transformer,RandomForestClassifier(random_state=42))
#pipe_svm_pca = make_pipeline( svm.SVC(column_transformer,random_state=42))

voting_classifier = VotingClassifier(estimators=[
    ('lgbm', classifier_lgbm), 
    ('xgb', classifier_xgb)], 
    #('rf', classifier_rf),
    #('svm', classifier_svm)],
                 voting='soft', n_jobs=-1)

In [53]:
oof_pred_lgbm = cross_val_predict(classifier_lgbm, 
                             train, 
                             train[target_column], 
                             cv=5,
                             method="predict_proba", verbose=20)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [ ]:
print("Cross validation AUC {:.4f}".format(roc_auc_score(train[target_column], oof_pred_lgbm[:,1])))
classifier_lgbm.fit(train, train[target_column])
test_preds_lgbm = classifier_lgbm.predict_proba(test)[:,1]

In [30]:
oof_pred_xgb = cross_val_predict(classifier_xgb, 
                             train, 
                             train[target_column], 
                             cv=5,
                             method="predict_proba", verbose=20)
print("Cross validation AUC {:.4f}".format(roc_auc_score(train[target_column], oof_pred_xgb[:,1])))
classifier_xgb.fit(train, train[target_column])
test_preds = classifier_xgb.predict_proba(test)[:,1]

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  7.1min remaining:    0.0s


KeyboardInterrupt: 

In [29]:
sub = pd.read_csv("data/sample_submission.csv")
sub.head()
sub[target_column] = test_preds
sub.to_csv("results/submit_results.csv", index=False)
print('done')

done
